# Composing Time Constructions

The current method for isolating phrase heads ([here](https://nbviewer.jupyter.org/github/ETCBC/heads/blob/master/phrase_heads.ipynb)) requires strenuous and ineloquent processing of BHSA subphrase relations. The subphrases are not always consistently encoded and suffer from numerous exceptional cases. The result is that the method is rather convoluted and ineloquent.

This notebook will explore the possibility of disconnecting semantic head analysis from the ETCBC subphrase encoding. 

A "semantic" head is the primary content word of a phrase, following Croft's "Primary Information Bearing Unit":

> **The noun and the verb are the PRIMARY INFORMATION_BEARING UNITS (PIBUs) of the phrase and clause respectively. In common parlance, they are the content words. PIBUs have major informational content that functional elements such as articles and [auxiliaries] do not have. (Croft, *Radical Construction Grammar*, 2001, 258; see also Shead, *Radical Frame Semantics and Biblical Hebrew*, 104)**

> **A (semantic) head is the profile equivalent that is the primary information-bearing unit, that is, the most contentful item that most closely profiles the same kind of thing that the whole constituent profiles. (ibid., 259)**

Croft also provides an additional criterion to "profile equivalence":

> **If the criterion of profile equivalence produces two candidates for headhood, the less schematic meaning is the PIBU; that is, the PIBU is the one with the narrower extension, in the formal semantic sense of that term (ibid., 259)**

## Inquiry

Can we isolate semantic phrase heads in BHSA using only the phrase_atom and phrase limits? This question indeed means that we  take the phrase_atom/phrase boundaries for granted. Empirically, the validity of BHSA phrase boundaries needs to be tested. But for now, the exercise of isolating semantic phrase heads could be seen as the first step towards reproducible phrase boundaries.

## Basic Concepts

A semantic head will most often stand in a syntactically independent position. For Hebrew nominal phrases, that essentially means a word which is not precided by a construct, and which is semantically central (excluding attributive slots (e.g. H + noun + H + ATTRIBUTIVE) or an adjectival slots (e.g. noun + noun as in אישׁ טוב).

Quantifier expressions present unique cases, which may be syntactically independent but semantically secondary. These are expressed through specialized lexical items such as cardinal numbers and qualitative quantifiers (e.g.  "כל" and "חצי").

Another complication is the use of nouns as prepositional items. Such uses can be seen with words like פני "face" such as לפני "in front," and even words like ראשׁ as in ראשׁ החדשׁ "beginning of the month." 

Other expressions of quantity, quality, and function provide similar complexities. These cases have to be specified in advance.

### Ambiguity

Considerable ambiguity is present in several of cases:

**`A B and C`**<br>
Given A, B, C == nominal words. Is their relationship `A // B // C` or `A+B // C`. In other words: **what is the relationship of two adjacent nominal words given a list?** Is B a descriptor of A or is it an independent element? 

**`A of B and C`**<br>
Is it, `(A of B) // (C)` or `(A of (B // C)`

Or even:

**`A of B C and D`**<br>
This pattern combines elements from both ambiguous cases.

To address these ambiguities we will apply a battery of disambiguation attempts. Some of those attempts will draw from corpus data, i.e. do we ever see `B and C` with the conjunction explicitly elsewhere in the corpus? Or do we ever see a `A of C` excplicitly in the corpus? Accents may also play a role: do we see a conjunctive or disjunctive accent between `B C`? 

## Prerequisites

A number of pre-defined word sets are needed for processing quantification and ambiguous adjacency. These sets are made available in the form of `wsets`, a dictionary containing word sets that are calculated in to the `wordsets` directory of this repository. The following wordsets have been defined:

* nominals – a set of word nodes with parts of speech and participles that have the potential to function as nominalized elements. The selected parts of speech are quite permissive: `{'subs', 'nmpr', 'adjv', 'advb', 'prde', 'prps', 'prin', 'inrg'}`. Since parts of speech are not taken as universal linguistic categories but only summaries of language-specific word tendencies (cf. Croft, *Radical Construction Grammar*, 2001), we consider that almost any part of speech can be used in a nominal pattern (or construction). There are some upper limits to this assumption, though. For instance, we exclude cojunctions, articles, prepositions, and negators. 
* prepositions – a word set consisting of words with a part of speech category of `prep`, a lexical set (`ls`) feature of `ppre` ("potential preposition"), as well as a select group of nouns like פני "face" which have been processed for prepositionality. 
* quantifiers - consists of word nodes that are cardinal numbers or qualitative quantifiers such as כל.
* mword – mapping from a word to its phonological word group ("masoretic word"); joins words on maqqeph and ø space
* accent_type – a mapping from a word to its accent type: conjunctive or disjunctive
* conj_pairs – a dict of observed conjunction pairings of lexemes in the corpus: `A & B`
* cons_pairs – a dict of observed construct pairings of lexemes in the corpus: `A of B`
* mom – mapping from word node to its mother word node for a specified relationship: `mom[A]['coord'] = B`
* kid – opposite of mom; mapping from word to its children nodes for a relationship: `kid[A]['cons'] = B`

**Let's get started**. We load the necessary functions and BHSA data (straight from source).

In [1]:
import sys
import collections
import pickle
import random
import re
import itertools
import copy
from IPython.display import display, HTML
from datetime import datetime
from pprint import pprint
from tf.app import use
from tf.fabric import Fabric
from tools.locations import data_locations

# load custom BHSA data + heads
TF = Fabric(locations=data_locations.values())
load_features = ['g_cons_utf8', 'trailer_utf8', 'label', 'lex',
                 'role', 'rela', 'typ', 'function', 'language',
                 'pdp', 'gloss', 'vs', 'vt', 'nhead', 'head', 
                 'mother', 'nu', 'prs', 'sem_set', 'ls', 'st',
                 'kind', 'top_assoc', 'number', 'obj_prep',
                 'embed', 'freq_lex', 'sp']
api = TF.load(' '.join(load_features))
F, E, T, L = api.F, api.E, api.T, api.L # shortform TF methods

A = use('bhsa', api=api, silent=True)
A.displaySetup(condenseType='phrase', withNodes=True, extraFeatures='st')

This is Text-Fabric 7.8.12
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

123 features found and 6 ignored
  0.00s loading features ...
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
  7.41s All features loaded/computed - for details use loadLog()


# Machinery

We could use some machinery to do the hard work of looking in and around a node. In the older approach we used TF search templates. But these are not very efficient at scale, and they are always bound by the limits of the query language. I take another approach here: a set of classes that specify locations and directions within a specified context.

In [2]:
from tools.langtools import Positions, Walker, Dummy

## `Positions`

The `Positions` class enables concise access to adjacent nodes within a given context. This allows us to write algorithms with query-like efficiency with all of the power of Python. 

This class is instantiated on a word node and can provide contextual look-up data for a given word. For example, given a phrase containing the following word nodes:

> (189681, 189682, **189683**, 189684, 189685, 189686) <br>

representing the following phrase (space separated for clarity):

> ב שׁנת **שׁלשׁים** ו שׁמנה שׁנה

Given that the bolded node, `189683` is our `source` word, we instantiate the class, feeding in the node, the "phrase_atom" string (which is the context we want to search within), and an instance of Text-Fabric (`tf`):

In [3]:
      #    source node    context  TF instance  
      #         |            |       |
P = Positions(189683, 'phrase_atom', A).get

If we want to obtain the word adjacent one space forward, we simply ask `P` for `1`, which gives us the next word in the phrase.

In [4]:
P(1)

189684

If we try to ask for 4 words forward, we go beyond the bounds of the phrase. But `P` handles this by returning nothing:

In [5]:
P(4)

To look back one word, we simply give a negative value:

In [6]:
P(-1)

189682

Finally, `P` can be used to quickly call features on these words. For instance, in order to get the lexeme of the word two words in front of `189683`:

In [7]:
P(2,'lex')

'CMNH/'

And if we want to get a number of features, we can just add other features to the arguments. The result is a feature set:

In [8]:
P(2, 'lex', 'nu')

{'CMNH/', 'sg'}

`P` can also handle features on the source node itself by giving a positionality of `0`:

In [9]:
P(0, 'lex')

'CLC/'

## `Walker`

`Walker` performs a similar function to `Positions`, except it is ambiguous to exact positions, walking either `ahead` or `back` from the source to a target node in the context. A function must be supplied that returns `True` on the target node.

We instantiate the `Walker` using the same source and context as above.

In [10]:
      #  source node    context  TF instance  
      #       |            |       |
Wk = Walker(189683, 'phrase_atom', A)

`Walker` is demonstrated below with the same word. A simple `lambda` function is used to test for the lexeme. In the example below, we find the first word ahead of `189683` that is a cardinal number:

In [11]:
Wk.ahead(lambda w: F.ls.v(w) == 'card')

189685

An alternative demonstrates the `None` returned on the lack of a valid match.

In [12]:
Wk.ahead(lambda w: F.ls.v(w) == 'BOOGABOOGA')

Another example wherein we walk backwards to the preposition:

In [13]:
Wk.back(lambda w: F.sp.v(w) == 'prep')

189681

We can also specify that the walk should be interrupted under certain conditions with a `stop` function. In this case we walk forward to the next cardinal number, but the walk is interrupted when the `stop` function detects a conjunction.

In [14]:
Wk.ahead(lambda w: F.ls.v(w) == 'card',
         stop=lambda w: F.sp.v(w) == 'conj')

We can also specify the opposite with a `go` function argument, which defines the nodes that allowed to intervene between `source` and `target`. Below we specify that *only* a conjunction should intervene.

In [15]:
Wk.ahead(lambda w: F.ls.v(w) == 'card',
         go=lambda w: F.sp.v(w) == 'conj')

189685

The `go` and `stop` functions can be as permissive or strict as desired.

Finally, we can tell `Walker` that the output of the validation function should be returned instead of the node itself with the optional argument `output=True`:

In [16]:
val_funct = lambda w: F.ls.v(w) if F.ls.v(w)=='card' else None

Wk.ahead(val_funct, output=True)

'card'

This ability is useful for certain tests.

## `Dummy`

When writing conditions and logic, we want an object that passively receives `NoneType`s or zero `int`s without throwing errors. Such an object should also return `None` to reflect its `False` value. `Dummy`, provides such functionality. `Dummy` can receive all of the arguments, kwargs, and function calls as a `Positions` or `Walker` object. But it returns absolutely nothing. Ouch.

In [17]:
D = Dummy(None, 'phrase_atom', A)

The function call below returns `None`:

In [18]:
D.get(1)

As does this:

In [19]:
D.get(1, 'lex')

And even this:

In [20]:
D.ahead(1)

`D` is essentially a souless void that consumes whatever you throw at it and gives nothing in return.

For safe-calls on a `Position` or `Walker` object, assign nodes to it via a function with a `Dummy` given on null nodes:

In [21]:
def getPos(node, context, tf):
    """A function to get Positions safely."""
    if node:
        return Positions(node, context, tf)
    else:
        return Dummy() # <- give dummy on empty node

So:

In [22]:
P = getPos(None, 'phrase_atom', A)
P.get(1)

Or:

In [23]:
P = getPos(1, 'phrase_atom', A)
P.get(1)

2

# Need for Semantic Data

The accurate processing of word connections depends on fuller semantic data than BHSA provides. Future semantic data could be stored in a similar way to word sets (`wsets`). 

For example, in the two phrases

> (Exod 25:39) ככר זהב טהור <br>
> (2 Sam 24:24) בכסף שקלים חמשׁים

we see that זהב and כסף, despite being in two different positions with two different words indicates a kind of "composed of" semantic concept: "round gold" (i.e. round composed of gold) and "silver shekels" (shekels composed of silver). To process these kinds of links, we need a list of nouns that often function as "material." But this is only the beginning. Many other words will have specific semantic values that motivate their syntactic behavior. Such a scope lies outside the bounds of this author's current project on Hebrew time phrases.

## A Compromise: Time Phrases

Since constructing these semantic classes is vastly time consuming, I want to start with a smaller set of cases. I will instead focus on parsing connections within time phrases for now. This is because I am analyzing time phrases in my current ongoing PhD project. 

In [24]:
def disjoint(ph):
    """Isolate phrases with gaps."""
    ph = L.d(ph,'word')
    for w in ph:
        if ph[-1] == w:
            break
        elif (ph[ph.index(w)+1] - w) > 1:
            return True

In [25]:
alltimes = [
    ph for ph in F.otype.s('timephrase') 
        if len(L.d(ph, 'word')) > 2
]
    
timephrases = [ph for ph in alltimes if not disjoint(ph)]

print(f'{len(timephrases)} phrases ready')

2102 phrases ready


## Search & Display Functions

The functions below allow for fast searching and displaying of queries using a `Construction` object, described in the next section.

In [26]:
def pretty(obj, condense='phrase', **kwargs):
    """Show a linguistic object that is not native to TF app."""
    index = kwargs.get('index')
    kwargs = {k:v for k,v in kwargs.items() if k not in {'index'}}
    show = L.d(obj, condense) if index is None else (L.d(obj, condense)[index],)
    print(show, not index, index)
    A.prettyTuple(show, seq=kwargs.get('seq', obj), **kwargs)

def prettyconds(cx):
    '''
    Iterate through an explain dict for a rela
    and print out all of checked conditions.
    '''
    for smallcx in cx.unfoldcxs():
        print(f'-- {smallcx} --')
        for case in smallcx.cases:
            for cond, value in case['conds'].items():
                print('{:<30} {:>30}'.format(cond, str(value)))
            print()
        
def showmatch(cx, **kwargs):
    '''
    Displays a match from a Grammar test.
    '''
    
    # get slots for display
    refslots = kwargs.get('refslots', cx.slots)
    showcontext = tuple(set(L.u(s, 'phrase')[0] for s in refslots))
    timephrase = L.u(list(refslots)[0], 'timephrase')[0]        
    
    if not refslots:
        print('NO SLOTS TO DISPLAY: GIVE ARG refslots')
        return None
    
    if not cx:
        print('NO MATCHES')
        print('-'*20)
        A.prettyTuple(showcontext, extraFeatures='sp st', withNodes=True, seq=f'{timephrase} -> {cx}')
        if kwargs.get('conds'):
            prettyconds(cx)
        return None

    colors = itertools.cycle(['pink', 'lightblue', 
                              'yellow', 'lightgreen'])
    highlights = {}
    role2color = {}
    
    for role, slots in cx.role2slots.items():
        color = next(colors)
        role2color[role] = color
        for slot in slots:
            highlights[slot] = color
    
    A.prettyTuple(
        showcontext, 
        extraFeatures='sp st', 
        withNodes=True, 
        seq=f'{timephrase} -> {cx}', 
        highlights=highlights
    )
    # reveal color meanings
    for role,color in role2color.items():
        colmean = '<div style="background: {}; text-align: center">{}</div>'.format(color, role)
        display(HTML(colmean))
    
    pprint(cx.unfoldroles(), indent=4)
    print()
    if kwargs.get('conds'):
        prettyconds(cx)
    display(HTML('<hr>'))
        
def test_search(cxtest, show=10, end=None, pattern=''):
    '''
    Searches phrases with the specified relation 
    and prints out their descriptive explanation.
    '''
    
    start = datetime.now()
    print('beginning search')
    
    # build a convenient test set of words
    phrases = list(F.otype.s('timephrase'))
    words = [w for ph in phrases for w in L.d(ph, 'word')]
    
    # random shuffle to get good diversity of examples
    random.shuffle(words)
    matches = []
    append = matches.append
    
    # iterate and find matches on words
    for i,w in enumerate(words):

        # update every 5000 iterations
        if i%5000 == 0:
            print(f'\t{len(matches)} found ({i}/{len(words)})')
        
        # run test for construction
        test = cxtest(w)
        
        # save results
        if test:
            if pattern:
                if test.pattern == pattern:
                    append(test)
            else:
                append(test)
            
        # stop at end
        if len(matches) == end:
            break
        
    # display
    print('done at', datetime.now() - start)
    print(len(matches), 'matches found...')
    print('showing', end)
    
    for match in matches[:show]:
        showmatch(match)

## Construction Classes

* `Construction` - an object that represents a linguistic construction; the class records roles and the words that occupy them, as well as has methods for accessing and retrieving data on embedded roles/other constructions
* `CXBuilder` - matches conditions to build `Construction` objects; populates them with requisite data

In [27]:
class Bunch(object):
    """Stores variables for shorthand and safe access.
    
    Like a dot-dictionary.
    """
    def __init__(self, vardict):
        """Initialize variables object with dict."""
        self.dict = vardict
        for k,v in vardict.items():
            setattr(self, k, v)
    def __getattr__(self, name):
        return None
    def __deepcopy__(self, memo=None):
        """Handle deep copy errors by instancing a diff Bunch object."""
        return Bunch({k:v for k,v in self.dict.items()})
    
class Construction(object):
    """A linguistic construction and its attributes."""
    
    def __init__(self, **specs):
        """Initialize construction item.
        
        **specs:
            name: A name for the construction.
            roles: A dict which maps roles
                to either another Construction item
                or to a Text-Fabric word node.
            cases: A tuple containing condition dicts
                that were evaluated when processing this
                Construction. Key is string containing condition,
                value is Boolean.
            conds: A condition dict containing all of the
                conditions that evaluated to True to validate
                this Construction.
        """
        for k,v in specs.items():
            setattr(self, k, v)
        self.match = specs.get('match', {})
        self.name = specs.get('name', '')
        self.kind = specs.get('kind', '')
        self.pattern = specs.get('pattern', specs.get('name', ''))
        self.roles = Bunch(specs.get('roles', {}))
        self.conds = specs.get('conds', {})
        self.cases = specs.get('cases', tuple())
        self.indexslots()
        
    def __bool__(self):
        """Determine truth value of CX."""
        if self.match:
            return True
        else:
            return False
        
    def __repr__(self):
        """Display CX name with slots."""
        if self:
            return f'CX {self.name} {self.slots}'
        else:
            return '{CX EMPTY}'
            
    def __eq__(self, other):
        """Determine slot/role-based equality between CXs."""
        if (
            self.slots2role == other.slots2role
            and self.name == other.name
        ):
            return True
        else:
            return False
        
    def __contains__(self, cx):
        """Determine whether certain CX is contained in this one."""
        return cx in list(self.unfoldcxs())

    def mapslots(self, rolesdict, rolename=None):
        """Recursively map all slots to top embedding role name.

        Match items contain a roles key which can contain
        any number of other match items. This function maps
        all constituent words (Text-Fabric "slots") to their
        top-level linguistic unit (linguistic role).
        """
        for role, item in rolesdict.items():
            if type(item) == Construction:
                self.mapslots(
                    item.roles.dict,
                    rolename=rolename or role
                )
            elif type(item) == int:
                self.role2slots[rolename or role].add(item)
                self.slots.add(item)   
            
    def indexslots(self):
        """Indexes slots contained in this CX."""
        self.role2slots = collections.defaultdict(set)
        self.slots = set()
        self.mapslots(self.roles.dict) # populates role2slots and slots
        self.slots = set(sorted(self.slots)) # sort slots
        self.slots2role = {
            tuple(sorted(slots)):role 
                for role, slots in self.role2slots.items()
        }  
      
    def unfoldroles(self, cx=None):
        """Return all contained construction roles as a dict.

        Recursively calls down into construction objects to convert
        to role.dict with TF slots.
        """
        cx = cx or self
        roledict={}
        roledict['__cx__'] = cx.name
        for role, item in cx.roles.dict.items():
            if type(item) == Construction :
                roledict[role] = self.unfoldroles(item)
            elif type(item) == int:
                roledict[role] = item
        return roledict
    
    def unfoldcxs(self, cx=None):
        """Return all contained constructions with flattened structure.
        
        Recursively calls down into construction objects and yields them.
        """
        cx = cx or self
        yield cx
        for role, item in cx.roles.dict.items():
            if type(item) == Construction :
                yield from self.unfoldcxs(item)  
    
    def unfoldcxpath(self, slots):
        """Return all contained constructions along a path.

        Recursively calls down into construction objects and yields them.
        """
        cx_path = []
        for cx in self.unfoldcxs():
            if set(slots).issubset(cx.slots):
                cx_path.append(cx)
        return cx_path
                
    def slots2cx(self, slottuple):
        """Return the embedded Construction to which a span of slots belong"""
        for cx in self.unfoldcxs():
            for slots, role in cx.slots2role.items():
                if slots == slottuple:
                    return cx
    
    def getslotrole(self, slot):
        """Returns the role to which a slot belongs to."""
        for role, slots in self.role2slots.items():
            if slot in slots:
                return role
                
    def updaterole(self, role, newitem):
        """Updates a role in the CX."""
        setattr(self.roles, role, newitem)
        self.roles.dict[role] = newitem
        self.indexslots() # remap slots

In [28]:
class CXbuilder(object):
    """Identifies and builds constructions using Text-Fabric nodes."""
    
    def __init__(self):
        """Initialize CXbuilder, giving methods for CX detection."""
        
        self.cache = collections.defaultdict(
            lambda: collections.defaultdict()
        )
        
        # NB: objects below should be overwritten 
        # and configured for the particular cxs needed
        self.precxs = {} # pre-requisite cxs that have been preprocessed
        self.cxs = tuple()
        self.yieldsto = {} 
    
    def debugmess(self, msg, toggle):
        """Prints debugging messages if toggled."""
        if toggle:
            sys.stderr.write(msg+'\n')
    
    def cxcache(self, element, name, method):
        """Get cx from cache or run."""
        try:
            return self.cache[element][name]
        except KeyError:
            return method(element)
    
    def test(self, *cases):
        """Populate Construction obj based on a cases's all Truth value.
        
        The last-matching case will be used to populate
        a Construction object. This allows more complex
        cases to take precedence over simpler ones.
        
        Args:
            cases: an arbitrary number of dictionaries,
                each of which contains a string key that
                describes the test and a test that evals 
                to a Boolean.
        
        Returns:
            a populated or blank Construction object
        """
        
        # find cases where all cnds == True
        test = [
            case for case in cases
                if all(case['conds'].values())
                    and all(case['roles'].keys())
        ]
        
        # return last test
        if test:
            cx = Construction(
                match=test[-1],
                cases=cases,
                **test[-1]
            )
            self.cache[cx.element][cx.name] = cx
            return cx
        else:
            return Construction(cases=cases, **cases[0])
        
    def findall(self, element):
        """Runs analysis for all constructions with an element.
        
        Returns as dict with test:result as key:value.
        """
        results = []
        
        # add pre-processed cxs
        for name, cx in self.precxs.get(element, {}).items():
            results.append(cx)
        
        # add cxs from this builder
        for funct in self.cxs:
            cx = funct(element)
            if cx:
                results.append(cx)
        return results
                        
    def sortbyslot(self, cxlist):
        """Sort constructions by order of contained slots."""
        sort = sorted(
            ((sorted(cx.slots), cx) for cx in cxlist),
            key=lambda k: k[0]
        )
        return [cx[-1] for cx in sort]
    
    def clusterCXs(self, cxlist):
        """Cluster constructions which overlap in their slots/roles.

        Overlapping constructions form a graph wherein the constructions 
        are nodes and the overlaps are edges. This algorithm retrieves all 
        interconnected constructions. It does so with a recursive check 
        for overlapping slot sets. Merging the slot sets produces new 
        overlaps. The algorithm passes over all constructions until no 
        further overlaps are detected.

        Args:
            cxlist: list of Construction objects

        Returns:
            list of lists, where each embedded list 
            is a cluster of overlapping constructions.
        """

        clusters = []
        cxlist = [i for i in cxlist] # operate on copy

        # iterate until no more intersections found
        thiscluster = [cxlist.pop(0)]
        theseslots = set(s for s in thiscluster[0].slots)

        # loop continues as it snowballs and picks up slots
        # loop stops when a complete loop produces no other matches
        while cxlist:

            matched = False # whether loop was successful

            for cx in cxlist:
                if theseslots & cx.slots:
                    thiscluster.append(cx)
                    theseslots |= cx.slots
                    matched = True

            # cxlist shrinks; when empty, it stops loop
            cxlist = [
                cx for cx in cxlist 
                    if cx not in thiscluster
            ]

            # assemble loop
            if not matched:
                clusters.append(thiscluster)
                thiscluster = [cxlist.pop(0)]
                theseslots = set(s for s in thiscluster[0].slots)
        
        # add last cluster
        clusters.append(thiscluster)

        return clusters

    def test_yield(self, cx1, cx2):
        """Determine whether to submit a cx1 to cx2."""
        
        # get name or class yields
        cx1yields = self.yieldsto.get(
            cx1.name,
            self.yieldsto.get(cx1.kind, set())
        )
        # test yields
        if type(cx1yields) == set:
            return bool({cx2.name, cx2.kind} & cx1yields)
        elif type(cx1yields) == bool:
            return cx1yields
           
    def weaveCX(self, cxlist, cx=None, debug=False):
        """Weave together constructions on their intersections.

        Overlapping constructions form a graph wherein constructions 
        are nodes and the overlaps are edges. The graph indicates
        that the constructions function together as one single unit.
        weaveCX combines all constructions into a single one. Moving
        from right-to-left (Hebrew), the function consumes and subsumes
        subsequent constructions to previous ones. The result is a 
        single unit with embedding based on the order of consumption.
        Roles in previous constructions are thus expanded into the 
        constructions of their subsequent constituents.
        
        For instance, take the following phrase in English:
        
            >    "to the dog"
            
        Say a CXbuilder object contains basic noun patterns and can
        recognize the following contained constructions:
        
            >    cx Preposition: ('prep', to), ('obj', the),
            >    cx Definite: ('art', the), ('noun', dog)
        
        When the words of the constructions are compared, an overlap
        can be seen:
        
            >    cx Preposition:    to  the
            >    cx Definite:           the  dog
        
        The overlap in this case is "the". The overlap suggests that
        the slot filled by "the" in the Preposition construction 
        should be expanded. This can be done by remapping the role
        filled by "the" alone to the subsequent Definite construction.
        This results in embedding:
        
            >    cx Preposition: ('prep', to), 
                                 ('obj', cx Definite: ('art', the), 
                                                      ('noun', dog))
        
        weaveCX accomplishes this by calling the updaterole method native
        to Construction objects. The end result is a list of merged 
        constructions that contain embedding.
        
        Args: 
            cxlist: a list of constructions pre-sorted for word order;
                the list shrinks throughout recursive iteration until
                the job is finished
            cx: a construction object to begin/continue analysis on
            debug: an option to display debugging messages for when 
                things go wrong 🤪
                
        Prerequisites:
            self.yieldsto: A dictionary in CXbuilder that tells weaveCX
                to subsume one construction into another regardless of
                word order. Key is name of submissive construction, value
                is a set of dominating constructions. Important for, e.g., 
                cases of quantification where a head-noun might be preceded 
                by a chain of quantifiers but should still be at the top of 
                the structure since it is more semantically prominent.
                
        Returns:
            a list of composed constructions
        """
        debugmess = self.debugmess
        
        debugmess(f'\nReceived {cx} with cxlist {cxlist}', debug)

        # the search is complete, stop here
        if not cxlist:
            debugmess(f'\tSearch complete with {cx} with roles: {cx.roles.dict}', debug)
            return cx
        
        # or no search necessary, stop here
        elif cx is None and len(cxlist) == 1:
            debugmess(f'\tSearch complete with {cxlist[0]} with roles: {cxlist[0].roles.dict}', debug)
            return cxlist[0]

        # Copy constructions and operate on copies
        cx1 = cx or copy.deepcopy(cxlist.pop(0))
        cx2 = copy.deepcopy(cxlist.pop(0))
        debugmess(f'\t comparing {cx1} & {cx2}', debug)

        # replace cx1 if already contained in cx2
        if (cx1 in cx2):
            debugmess(f'\t Discarding cx1 because cx2 already contains it...', debug)
            return self.weaveCX(cxlist, cx2, debug=debug)

        # get first slot of intersection between cx1 and 2
        # get that slot's role in both cxs
        link = tuple(sorted(cx1.slots & cx2.slots))
        debugmess(f'\t link is {link}', debug)

        # retrieve lowest-contained construction with link
        cx1link = cx1.slots2cx(link)
        debugmess(f'\t\t cx1link is {cx1link}', debug)

        # submit cx1 to cx2 if cx2 is semantically dominant
        if self.test_yield(cx1link, cx2):

            debugmess(f'\t cx2 is semantically dominant over cx1...', debug)

            link1path = list(cx1.unfoldcxpath(cx1link.slots))[:-1]
            debugmess(f'\t\t searching {link1path}', debug)

            # submit until cx2's dominance ends
            while (
                link1path
                and self.test_yield(link1path[-1], cx2)
            ):
                cx1link = link1path.pop()

            debugmess(f'\t\t cx1link iterated upward to {cx1link}', debug)

            # subsume cx1 to cx2
            debugmess(f'\t\t cx2 role [{cx2.slots2role[link]}] remapping to {cx1link}...', debug)
            cx2.updaterole(cx2.slots2role[link], cx1link)
            debugmess(f'\t\t remapping done with {cx2} containing roles {cx2.roles.dict}', debug)     

            # subsume cx2 to enclosing cx
            bigcx = link1path[-1] if link1path else None 
            if bigcx:
                debugmess(f'\t assigning cx2 {cx2} to bigcx {bigcx}', debug)
                
                #return self.weaveCX([cx2]+cxlist, bigcx, debug=debug)
                
                biglink = tuple(sorted(cx1link.slots & bigcx.slots))
                bigrole = bigcx.slots2role[biglink]
                debugmess(f'\t\t big role [{bigrole}] remapping to {cx2}...', debug)
                bigcx.updaterole(bigrole, cx2)
                cx1.indexslots()
                return self.weaveCX(cxlist, cx1, debug=debug)

            # or continue with cx2
            else:
                debugmess(f'\t\t moving on with cx2 as new primary: {cx2}', debug)
                return self.weaveCX(cxlist, cx2, debug=debug)
        
        # submit cx2 to cx1
        else:
            linkcx1 = cx1.slots2cx(link)
            debugmess(f'\t submitting {cx2} to {linkcx1}...', debug)
            linkrole1 = linkcx1.slots2role[link]
            debugmess(f'\t\t role [{linkrole1}] remapping to {cx2}...', debug)
            linkcx1.updaterole(linkrole1, cx2)
            cx1.indexslots()
            debugmess(f'\t\t remapping done with {linkcx1} containing roles {linkcx1.roles.dict}', debug)
            return self.weaveCX(cxlist, cx1, debug=debug)
    
    def analyzestretch(self, stretch, debug=False):
        """Analyze an entire stretch of a linguistic unit.
        
        Applies construction tests for every constituent 
        and merges all overlapping constructions into a 
        single construction.
        
        Args:
            stretch: an iterable containing elements that
                are tested by construction tests to build
                Construction objects. e.g. stretch might be 
                a list of TF word nodes.
            debug: option to display debuggin messages
        
        Returns:
            list of merged constructions
        """
        
        # match elements to constructions based on tests
        rawcxs = [
            match for element in stretch
                for match in self.findall(element)
                    if match
        ]
        
        # return empty results
        if not rawcxs:
            return []
            
        # cluster and sort matched constructions
        clsort = [
            self.sortbyslot(cxlist)
                for cxlist in self.clusterCXs(rawcxs)    
        ]
    
        # merge overlapping constructions
        cxs = [
            self.weaveCX(cluster, debug=debug)
                for cluster in clsort
        ]
        
        return cxs

## Word Constructions

The `wordConstructions` builder class recognizes word semantic classes and types based on provided criteria.

In [29]:
class wordConstructions(CXbuilder):
    """Build word constructions."""
    
    def __init__(self, tf, **kwargs):
        
        """Initialize with Constructions attribs/methods."""
        CXbuilder.__init__(self)
        
        # set up TF data for tests
        self.tf = tf
        self.F, self.T, self.L = tf.api.F, tf.api.T, tf.api.L
        self.context = kwargs.get('context', 'phrase_atom')
        
        # Order matters! More specific meanings first
        self.cxs = (
            self.cont,
            self.prep,
            self.quant,
            self.ordn,
        )
        self.kind = 'word_cx'
        
    def getP(self, node):
        """Get Positions object for a TF node.
        
        Return Dummy object if not node.
        """
        if not node:
            return Dummy()
        return Positions(node, self.context, self.tf).get
    
    def getWk(self, node):
        """Get Walker object for a TF word node.
        
        Return Dummy object if not node.
        """
        if not node:
            return Dummy()
        return Walker(node, self.context, self.tf)
    
    def cont(self, w):
        """A content word.
        
        A content word covers a wide range of
        traditional parts of speech, namely,
            [nouns, proper nouns, adjectives, adverbs]
        In traditional terms, all of these words that have 
        potential to function as 'nouns'. Notably,
        this set includes participial instances of verbs.
        """
        
        F = self.F
        
        return self.test(
            {
                'element': w,
                'name': 'cont',
                'kind': self.kind,
                'pattern': 'pos',
                'roles': {'head': w},
                'conds': {
                    'F.sp.v(w) in subs|nmpr|adjv|advb':
                        F.sp.v(w) in {
                            'subs', 'nmpr',
                            'adjv', 'advb',
                        },
                    'not preposition(w)':
                        not self.cxcache(
                            w,
                            'prep',
                            self.prep
                        ),
                }
            },
            {
                'element': w,
                'name': 'content',
                'kind': self.kind,
                'pattern': 'participle',
                'roles': {'head': w},
                'conds': {
                    'F.sp.v(w) == verb':
                        F.sp.v(w) == 'verb',
                    'F.vt.v(w) in {ptcp, ptca}':
                        F.vt.v(w) in {'ptcp', 'ptca'},
                }
            },
        )    

    def prep(self, w):
        """A preposition word."""
        
        P = self.getP(w)
        F = self.F
        name = 'prep'
        roles = {'head': w}
        return self.test(
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'ETCBC pdp',
                'roles': roles,
                'conds': {
                    'F.pdp.v(w) == prep':
                        F.pdp.v(w) == 'prep',
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'ETCBC ppre words',
                'roles': roles,
                'conds': {
                    'F.ls.v(w) == ppre':
                        F.ls.v(w) == 'ppre',
                    'F.lex.v(w) != DRK/':
                        F.lex.v(w) != 'DRK/',
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'R>C/',
                'roles': roles,
                'conds': {
                    'F.lex.v(w) == R>C/':
                        F.lex.v(w) == 'R>C/',
                    'F.st.v(w) == c':
                        F.st.v(w) == 'c',
                    'P(-1,pdp) == prep':
                        P(-1,'pdp') == 'prep',
                    'phrase is adverbial':
                        F.function.v(
                            L.u(w,'phrase')[0]
                        ) in {
                            'Time', 'Adju', 
                            'Cmpl', 'Loca',
                        },
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'construct lexs',
                'roles': roles,
                'conds': {
                    'F.lex.v(w) in lexset':
                        F.lex.v(w) in {
                            'PNH/','TWK/', 
                            'QY/', 'QYH=/', 
                            'QYT/', '<WD/'
                        },
                    'F.prs.v(w) == absent':
                        F.prs.v(w) == 'absent',
                    'F.st.v(w) == c':
                        F.st.v(w) == 'c'
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'L+BD',
                'roles': roles,
                'conds': {
                    'F.lex.v(w) == BD/':
                        F.lex.v(w) == 'BD/',
                    'P(-1,lex) == L':
                        P(-1,'lex') == 'L',
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': '>XRJT/',
                'roles': roles,
                'conds': {
                    'F.lex.v(w) == >XRJT/':
                        F.lex.v(w) == '>XRJT/',
                    'F.st.v(w) == c':
                        F.st.v(w) == 'c',
                    'P(1,lex) or P(2,lex) not >JWB|RC</':
                        not {
                            P(1,'lex'), P(2,'lex')
                        } & {
                            '>JWB/', 'RC</'
                        }
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': '<YM/ time',
                'roles': roles,
                'conds': {
                    'F.lex.v(w) == <YM/':
                        F.lex.v(w) == '<YM/',
                    'F.st.v(w) == c':
                        F.st.v(w) == 'c',
                    'F.function.v(phrase) == Time':
                        F.function.v(
                            L.u(w,'phrase')[0]
                        ) == 'Time',
                }
            }
        )
    
    def quant(self, w):
        """A quantifier word."""
        
        F = self.F
        P = self.getP(w)
        name = 'quant'
        roles = {'head': w}
        
        return self.test(
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'cardinal',
                'roles': roles,
                'conds': {
                    'F.ls.v(w) == card':
                        F.ls.v(w) == 'card',
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'qualitative',
                'roles': roles,
                'conds': {
                    'F.lex.v(w) in lexset':
                        F.lex.v(w) in {
                            'KL/', 'M<V/', 'JTR/',
                            'XYJ/', 'C>R=/', 'MSPR/', 
                            'RB/', 'RB=/',
                        },
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'portion',
                'roles': roles,
                'conds': {
                    'F.lex.v(w) in lexset':
                        F.lex.v(w) in {
                            'M<FR/', '<FRWN/',
                            'XMJCJT/',
                        },
                }
            },
        )
    
    def ordn(self, w):
        """An ordinal word."""
        
        F = self.F
        P = self.getP(w)
        roles = {'head': w}
        
        return self.test(
            {
                'element': w,
                'name': 'ordn',
                'kind': self.kind,
                'pattern': 'ETCBC ls',
                'roles': roles,
                'conds': {
                    'F.ls.v(w) == ordn':
                        F.ls.v(w) == 'ordn',
                }
            },
        )
    
    def cxdict(self):
        """Map all TF words to constructions.
        
        Method returns a dictionary of cxname:cx
        mappings. This enables efficient processing 
        when CXs are used in other CXbuilders.
        """
        slot2name2cx = collections.defaultdict(
            lambda: collections.defaultdict()
        )
        
        for w in self.F.otype.s('word'):
            for cx in self.findall(w):
                slot2name2cx[w][cx.name] = cx
                
        return slot2name2cx

## "TP" Constructions

The `TPConstructions` class prepares Time Phrase constructions.

In [30]:
class TPConstructions(CXbuilder):
    """Class for building time phrase constructions."""
    
    def __init__(self, wordcxs, tf, **kwargs):
        
        """Initialize with Constructions attribs/methods."""
        CXbuilder.__init__(self)
        
        # set up TF data for tests
        self.tf = tf
        self.F, self.T, self.L = tf.api.F, tf.api.T, tf.api.L
        self.context = kwargs.get('context', 'phrase_atom')
        self.precxs = wordcxs
        
        # set up convenient word cx references with Bunch object
        self.words = {
            s:Bunch(cxs) 
                for s, cxs in wordcxs.items()
        }
        # populate self.words with blanks
        # needed for null-matches without key error
        for w in self.F.otype.s('word'):
            if w not in self.words:
                self.words[w] = Bunch({})
        self.words[None] = Bunch({})
        
        # map cx searches for full analyses
        self.cxs = (
            self.defi,
            self.card_chain,
            self.demon,
            self.adjv,
            self.advb,
            self.attrib,
            self.geni,
            self.numb,
            self.prep,
        )
        
        # submit these cxs to cx in set 
        self.yieldsto = {
            'card_chain': {'numb_ph'},
            'word_cx': True
        }
        
        self.kind = 'TP_construction'

    def getP(self, node):
        """Get Positions object for a TF node.
        
        Return Dummy object if not node.
        """
        if not node:
            return Dummy()
        return Positions(node, self.context, self.tf).get
    
    def getWk(self, node):
        """Get Walker object for a TF word node.
        
        Return Dummy object if not node.
        """
        if not node:
            return Dummy()
        return Walker(node, self.context, self.tf)
        
    def defi(self, w):
        """Matches a definite construction."""
        
        P = self.getP(w)
        
        return self.test( 
            {
                'element': w,
                'name': 'defi_ph',
                'kind': self.kind,
                'roles': {'art': w, 'head': P(1)},
                'conds': {

                    'F.sp.v(w) == art':
                        self.F.sp.v(w) == 'art',

                    'bool(P(1))':
                        bool(P(1))
                }
            }
        )
    
    def prep(self, w):
        """Matches a preposition with a modified element."""
                
        P = self.getP(w)
        
        return self.test(
            {
                'element': w,
                'name': 'prep_ph',
                'kind': self.kind,
                'roles': {'prep':w, 'head': P(1)},
                'conds': {

                    'bool(w.prep)':
                        bool(self.words[w].prep),

                    'F.prs.v(w) == absent':
                        self.F.prs.v(w) == 'absent',
                    
                    'bool(P(1))':
                        bool(P(1)),
                }
            }
        )
        
    def geni(self, w):
        """Queries for "genitive" relations on a word."""
        
        P = self.getP(w)
        
        return self.test(
            {
                'element': w,
                'name': 'geni_ph',
                'kind': self.kind,
                'roles': {'geni': P(0), 'head': P(-1)},
                'conds': {

                    'P(-1, st) == c': 
                        P(-1,'st') == 'c',

                    'not (P(-1).quant)':
                        not (
                            self.words[P(-1)].quant
                        ),
                    
                    'not (P(-1).prep)':
                        not (
                            self.words[P(-1)].prep
                        ),
                }
            }
        )

    def advb(self, w):
        """Match and adverb and its mod."""
        
        P = self.getP(w)
        
        return self.test(
           {
                'element': w,
                'name': 'advb_ph',
                'kind': self.kind,
                'roles': {'advb': w, 'head': P(1)},
                'conds': {
                    'F.sp.v(w) == advb':
                        self.F.sp.v(w) == 'advb',
                    'P(-1,sp) != art':
                        P(-1,'sp') != 'art',
                    'bool(P(1))':
                        bool(P(1)),
                    'P(1,sp) != conj': # ensure not a nominal use
                        P(1,'sp') != 'conj',
                    'not (P(-1).prep)': # ensure not nominal
                        not (
                            self.words[P(-1)].prep
                        )
                }
            }
        )
    
    def adjv(self, w):
        """Matches a word serving as an adjective."""
        
        P = self.getP(w)
        F = self.F
        wd = self.words
        name = 'adjv_ph'
        
        # check for recursive adjective matches 
        a2match = self.adjv(P(-1)) if P(-1) else Construction()
        a2match_head = a2match.roles.head
        
        common = {
            
            'not (w.quant)':
                not (
                    wd[w].quant
                ),
            
            'bool(P(-1).cont)':
                bool(
                    wd[P(-1)].cont
                ),
            
            'P(-1, st) & {NA, a}': 
                P(-1,'st') in {'NA', 'a'},   
            
            'not (P(-1).quant)':
                not (
                    wd[P(-1)].quant
                ),
            
            'not (P(-1).prep)':
                not (
                    wd[P(-1)].prep
                )
        }
                
        tests = (
            
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'adjv (1x)',
                'roles': {'adjv':w, 'head': P(-1)},
                'conds': dict(common, **{
                    'F.sp.v(w) in {adjv, verb}':
                        F.sp.v(w) in {'adjv', 'verb'},
                })
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'adjv (2x)',
                'roles': {'adjv': P(0), 'head': a2match_head},
                'conds': dict(common, **{
                    
                    'P(0,sp) in {adjv, verb}':
                        P(0,'sp') in {'adjv', 'verb'},
                    
                     'self.adjv(P(-1)) and target != P(0)':
                        bool(a2match) and a2match_head != P(0)
                })
            }
        )

        return self.test(*tests)
     
    def attrib(self, w):
        """Identify elements in a attrib construction.
        
        In Hebrew this construction typically consists of four slots:
            > ה + A + ה + B
        Attrib identifies each of these elements and labels them.
        A is assumed to be the head, or modified, element and B
        is assumed to be an adjectival element.
        """
                
        # CX consists of two constituent cxs
        # start walk from head of first match
        defi1 = self.defi(w)
        d1head = defi1.roles.head
        Wk = self.getWk(d1head)

        # walk to next valid defi match
        # and allow adjectives to intervene:
        defi2 = Wk.ahead(
            lambda n: self.defi(n),
            go=lambda n: self.F.sp.v(n)=='adjv',
            output=True
        ) if Wk else Construction()
        defi2 = defi2 or Construction()
                
        return self.test(
            {
                'element': w,
                'name': 'attrib_ph',
                'kind': self.kind,
                'roles': {'head': defi1, 'attrib': defi2},
                'conds': {
                    'bool(defi1)':
                        bool(defi1),
                    'bool(defi2)':
                        bool(defi2), 
                }
            }
        )
        
    def numb(self, w):
        """Defines numerical relations with an non-quant word.
        
        Often but not always indicates quantification as other
        semantic relations are possible.
        """

        P = self.getP(w)
        Wk = self.getWk(w)
        wd = self.words
        is_nom = (
            lambda n:
                bool(wd[n].cont) and not wd[n].quant
        )
        behind_nom = Wk.back(is_nom, stop=lambda n: not is_nom(n)) 
        
        return self.test(
        
            {
                'element': w,
                'name': 'numb_ph',
                'kind': self.kind,
                'pattern': 'numbered forward',
                'roles': {'numb': w, 'head': P(1)},
                'conds': {
                    
                    'bool(w.quant)':
                        bool(wd[w].quant),
                    
                    'bool(P(1))':
                        bool(P(1)),
                    
                    'P(1,sp) != conj':
                        P(1,'sp') != 'conj',
                    
                    'not (P(1).quant)':
                        not(
                            wd[P(1)].quant
                        ),
                    
                    'not (P(1).prep)':
                        not(
                            wd[P(1)].quant
                        ),
                    
                    'P(-1,sp) != art':
                        P(-1,'sp') != 'art',
                },
            },  
            {
                'element': w,
                'name': 'numb_ph',
                'kind': self.kind,
                'pattern': 'numbered backward',
                'roles': {'numb': w, 'head': behind_nom},
                'conds': {
                    
                    'bool(w.quant)':
                        bool(wd[w].quant),
                    
                    'not Wk.ahead(is_nominal)':
                        not Wk.ahead(is_nom),
                    
                    'bool(Wk.back(is_nominal))':
                        bool(behind_nom),
                    
                    'F.st.v(behind_nom) in {a, NA}':
                        self.F.st.v(behind_nom) in {'a', 'NA'},
                }
            }
        )
        
    def card_chain(self, w):
        """Defines cardinal number chain constructions"""
        
        P = self.getP(w)
        F = self.F
        
        return self.test(
            {
                'element': w,
                'name': 'card_chain',
                'kind': self.kind,
                'pattern': 'adjacent',
                'roles': {'card':w, 'head':P(-1)},
                'conds': {
                    
                    'F.ls.v(w) == card':
                        F.ls.v(w) == 'card',
                    'P(-1,ls) == card':
                        P(-1,'ls') == 'card',                    
                }
            },
            {
                'element': w,
                'name': 'card_chain',
                'kind': self.kind,
                'pattern': 'conjunctive',
                'roles': {'card': w, 'head': P(-2), 'conj': P(-1)},
                'conds': {
                    'F.ls.v(w) == card':
                        F.ls.v(w) == 'card',
                    'P(-1,lex) == W':
                        P(-1,'lex') == 'W',
                    'P(-2,ls) == card':
                        P(-2,'ls') == 'card',   
                }
            }
        )
    
    def demon(self, w):
        """Defines an adjacent demonstrative construction."""
        
        P = self.getP(w)
        wd = self.words
        F = self.F
        name = 'demon_ph'
        
        return self.test(
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'adjacent forward',
                'roles': {'demon': w, 'head': P(1)},
                'conds': {
                    'prde in {F.pdp.v(w), F.sp.v(w)}':
                        'prde' in {F.pdp.v(w), F.sp.v(w)},
                    
                    'P(-1,sp) != art': # ensure not part of attrib pattern
                        P(-1,'sp') != 'art',
                    
                    'not (P(-1).prep)':
                        not(
                            wd[P(-1)].prep
                        ),
                    
                    'bool(P(1))':
                        bool(P(1)),
                    
                    'bool(P(1).cont)':
                        bool(wd[P(1)].cont),
                }
            },
            {
                'element': w,
                'name': name,
                'kind': self.kind,
                'pattern': 'adjacent back',
                'roles': {'demon': w, 'head': P(-1)},
                'conds': {
                    'prde in {F.pdp.v(w), F.sp.v(w)}':
                        'prde' in {F.pdp.v(w), F.sp.v(w)},
                    
                    'not (P(-1).prep)': # ensure not a part of particle
                        not(
                            wd[P(-1)].prep
                        ),
                    
                    'not (P(-1).quant)':
                        not(
                            wd[P(-1)].quant
                        ),
                    
                    'P(-1,sp) == subs':
                        P(-1,'sp') == 'subs',
                }
            }
        )

## Tests and Development

### Load Constructions

In [ ]:
words = wordConstructions(A) # word CX builder

# analyze all matches; return as dict
start = datetime.now()
print(f'Beginning word construction analysis...')
wordcxs = words.cxdict()
print(f'\t{datetime.now() - start} COMPLETE [{len(wordcxs)}] cxs loaded.')

Beginning word construction analysis...


In [ ]:
# time phrase CX builder
tpc = TPConstructions(wordcxs, A)

### Small Tests

In [167]:
# test_small = words.preposition(1)
# showmatch(test_small, conds=True)

### Stretch Tests

In [176]:
# test = tpc.analyzestretch(L.d(1446832, 'word'), debug=True)

# for res in test:
#     showmatch(res, conds=True)

### Pattern Searches

In [171]:
#test_search(words.ordinal, pattern='', show=100, end=10)

### Testing on Random Phrases

In [172]:
shuff = [k for k in timephrases
            if len(L.d(k,'word')) > 4]
random.shuffle(shuff)

In [173]:
for phrase in shuff[:50]:
    
    print('analyzing', phrase)
    elements = L.d(phrase,'word')
    
    try:
        cxs = tpc.analyzestretch(elements)
        if cxs:
            for cx in cxs:
                showmatch(cx, refslots=elements)
        else:
            showmatch(Construction(), refslots=elements)
    
    except:
        sys.stderr.write(f'\nFAIL...running with debug...\n')
        pretty(phrase)
        tpc.analyzestretch(elements, debug=True)
        raise Exception('...debug complete...')

analyzing 1450153


{   '__cx__': 'prep_ph',
    'head': {   '__cx__': 'attrib_ph',
                'attrib': {   '__cx__': 'defi_ph',
                              'art': 365657,
                              'head': {'__cx__': 'ordn', 'head': 365658}},
                'head': {   '__cx__': 'defi_ph',
                            'art': 365655,
                            'head': {'__cx__': 'cont', 'head': 365656}}},
    'prep': {'__cx__': 'prep', 'head': 365654}}



analyzing 1450170


{   '__cx__': 'prep_ph',
    'head': {'__cx__': 'cont', 'head': 366830},
    'prep': {'__cx__': 'prep', 'head': 366829}}



{   '__cx__': 'prep_ph',
    'head': {'__cx__': 'cont', 'head': 366832},
    'prep': {'__cx__': 'prep', 'head': 366831}}



...

{   '__cx__': 'prep_ph',
    'head': {   '__cx__': 'geni_ph',
                'geni': {   '__cx__': 'attrib_ph',
                            'attrib': {   '__cx__': 'defi_ph',
                                          'art': 92319,
                                          'head': {   '__cx__': 'cont',
                                                      'head': 92320}},
                            'head': {   '__cx__': 'defi_ph',
                                        'art': 92317,
                                        'head': {   '__cx__': 'cont',
                                                    'head': 92318}}},
                'head': 92316},
    'prep': {'__cx__': 'prep', 'head': 92315}}



analyzing 1449666


{   '__cx__': 'prep_ph',
    'head': {   '__cx__': 'attrib_ph',
                'attrib': {'__cx__': 'defi_ph', 'art': 308625, 'head': 308626},
                'head': {   '__cx__': 'defi_ph',
                            'art': 308623,
                            'head': {'__cx__': 'cont', 'head': 308624}}},
    'prep': {'__cx__': 'prep', 'head': 308622}}



analyzing 1450348


{   '__cx__': 'prep_ph',
    'head': {   '__cx__': 'attrib_ph',
                'attrib': {'__cx__': 'defi_ph', 'art': 387686, 'head': 387687},
                'head': {   '__cx__': 'defi_ph',
                            'art': 387684,
                            'head': {'__cx__': 'cont', 'head': 387685}}},
    'prep': {'__cx__': 'prep', 'head': 387683}}



### Testing on All Timephrases

In [175]:
phrase2cxs = collections.defaultdict(list)
nocxs = []

# time it
start = datetime.now()

print(f'{datetime.now()-start} beginning analysis...')

for i, phrase in enumerate(timephrases):
     
    # analyze all known relas
    elements = L.d(phrase,'word')
    
    # analyze with debug exceptions
    try:
        cxs = tpc.analyzestretch(elements)
    except:
        sys.stderr.write(f'\nFAIL...running with debug...\n')
        pretty(phrase)
        tpc.analyzestretch(elements, debug=True)
        raise Exception('...debug complete...')

    # save those phrases that have no matching constructions
    if not cxs:
        nocxs.append(phrase)
    else:
        phrase2cxs[phrase] = cxs
        
    # report status
    if i % 500 == 0 and i:
        print(f'\t{datetime.now()-start}\tdone with iter {i}/{len(timephrases)}')
        
print(f'{datetime.now()-start}\tCOMPLETE')
print('-'*20)
print(f'{len(phrase2cxs)} phrases matched with Constructions...')
print(f'{len(nocxs)} phrases not yet matched with Constructions...')

NameError: name 'wsets' is not defined

## Closing Gaps

### Identify Gaps

Find timephrases that contain un-covered words besides waw conjunctions.

In [354]:
gapped = []

for ph, cxs in phrase2cxs.items():
    
    ph_slots = set(
        s for s in L.d(ph,'word')
            if F.sp.v(s) != 'conj'
    )
    cx_slots = set(
        s for cx in cxs
            for s in cx.slots
                if F.sp.v(s) != 'conj'
    )
    
    if ph_slots.difference(cx_slots):
        gapped.append(cxs)
        
print(f'{len(gapped)} gapped phrases logged...')

140 gapped phrases logged...


In [355]:
for gp in gapped[:25]:
    for cx in gp:
        showmatch(cx)

{   '__cx__': 'prep',
    'head': {'__cx__': 'defi', 'art': 5150, 'head': 5151},
    'prep': 5149}



{   '__cx__': 'prep',
    'head': {'__cx__': 'geni', 'geni': 6216, 'head': 6215},
    'prep': 6214}



{'__cx__': 'geni', 'geni': 6218, 'head': 6217}



{'__cx__': 'geni', 'geni': 6221, 'head': 6220}



{'__cx__': 'geni', 'geni': 6224, 'head': 6223}



...

{'__cx__': 'prep', 'head': 96158, 'prep': 96157}



{'__cx__': 'prep', 'head': 102412, 'prep': 102411}



{'__cx__': 'prep', 'head': 103949, 'prep': 103948}



{'__cx__': 'prep', 'head': 104026, 'prep': 104025}



{'__cx__': 'prep', 'head': 114040, 'prep': 114039}

